In [1]:
def ichimoku(df):
    # Conversion line (tenkan)
    period9_high = df['MID_HIGH'].rolling(window=9).max()
    period9_low = df['MID_LOW'].rolling(window=9).min()
    df['conv'] = (period9_high + period9_low) / 2

    # Base line (kijun)
    period26_high = df['MID_HIGH'].rolling(window=26).max()
    period26_low = df['MID_LOW'].rolling(window=26).min()
    df['base'] = (period26_high + period26_low) / 2

    # Leading A (senkou span A, cloud)
    df['cloud1'] = ((df['conv']+df['base'])/2).shift(25)

    # Leading B (senkou span B, cloud)
    period52_high = df['MID_HIGH'].rolling(window=52).max()
    period52_low = df['MID_LOW'].rolling(window=52).min()
    df['cloud2'] = ((period52_high + period52_low) / 2).shift(25)
    # this means the first 76 rows would have NA value under cloud2 column

    # Leading A (senkou span A, future)
    df['lead1'] = (df['conv']+df['base'])/2

    # Leading B (senkou span B, future)
    df['lead2'] = (period52_high + period52_low) / 2

    # Lag (chikou)
    # df['lag'] = df['MID_PRICE'].shift(-24)
    # instead of comparing lagged line with historical price
    # I shift the historical price foward to compare with the currency price
    df['hist_price'] = df['MID_PRICE'].shift(25)

    df = df.dropna()
    df = df.drop(['MID_HIGH','MID_LOW'], axis=1)
    df['signal'] = 'flat'
    
    return df


In [2]:
def add_signal(df):
    for i in df.index:
        if all([df.loc[i]['conv'] > df.loc[i]['base'],
                df.loc[i]['lead1'] > df.loc[i]['lead2'],
                df.loc[i]['MID_PRICE'] > df.loc[i]['cloud1'],
                df.loc[i]['cloud1'] > df.loc[i]['cloud2'],
                df.loc[i]['MID_PRICE'] > df.loc[i]['hist_price']]):
            df.loc[i,['signal']] = 'long'
        else: continue

    # check for short signal
    for i in df.index:
        if all([df.loc[i]['conv'] < df.loc[i]['base'],
                df.loc[i]['lead1'] < df.loc[i]['lead2'],
                df.loc[i]['MID_PRICE'] < df.loc[i]['cloud1'],
                df.loc[i]['cloud1'] < df.loc[i]['cloud2'],
                df.loc[i]['MID_PRICE'] < df.loc[i]['hist_price']]):
            df.loc[i,['signal']] = 'short'
        else: continue
        
    return df

In [3]:
import refinitiv.data as rd
import pandas as pd

rd.open_session()
data = []

# security = ['JPY=', 'AUDJPY=', 
#            'AUD=', 'AUDNZD=']

security = ['JPY=', 'AUDJPY=', 'NZDJPY=', 'EURJPY=', 'GBPJPY=', 'CADJPY=', 'CHFJPY=',
           'AUD=', 'NZD=', 'EUR=', 'GBP=', 'CAD=', 'AUDNZD=', 'EURCHF=', 'EURGBP=']

date_0 = '2023-12-01'
date_1 = '2024-12-01'

for ccy in security:
    markets = rd.get_history(universe=[ccy], fields=['MID_OPEN', 'MID_HIGH', 'MID_LOW','MID_PRICE'],
                             interval="1h", start=date_0, end=date_1
                        )

    df = markets
    df = ichimoku(df)
    df = add_signal(df)
    
    entry = 0
    position = 0
    pnl = []

    for i in df.index:
        if df.loc[i]['signal'] == 'long':
            if position == 0:
                entry = df.loc[i]['MID_OPEN']
                position = 1
                continue
            elif position == -1:
                pnl.append(df.loc[i]['MID_PRICE'] - entry)
                entry = 0
                position = 0
                continue
            elif df.loc[i]['MID_PRICE'] < df.loc[i]['base']:
                pnl.append(df.loc[i]['MID_PRICE'] - entry)
                entry = 0
                position = 0
                continue
            else: continue
        elif df.loc[i]['signal'] == 'short':
            if position == 0:
                entry = df.loc[i]['MID_OPEN']
                position = -1
                continue
            elif position == 1:
                pnl.append(entry - df.loc[i]['MID_PRICE'])
                entry = 0
                position = 0
                continue
            elif df.loc[i]['MID_PRICE'] > df.loc[i]['base']:
                pnl.append(entry - df.loc[i]['MID_PRICE'])
                entry = 0
                position = 0
                continue
            else: continue
        elif df.loc[i]['signal'] == 'flat':
            if position == 0:
                continue
            elif position == 1:
                pnl.append(df.loc[i]['MID_PRICE'] - entry)
                entry = 0
                position = 0
                continue
            else: 
                pnl.append(entry - df.loc[i]['MID_PRICE'])
                entry = 0
                position = 0
                continue
                
    if 'JPY' in ccy:
        data.append([ccy, round(sum(pnl)*100,2), len(pnl), round(min(pnl)*100,2), 
                     round(max(pnl)*100,2) ,
                    round(sum(x>0 for x in pnl)/len(pnl),2), 
                    abs(round(sum(x for x in pnl if x>0)/sum(x for x in pnl if x<0),2))])
    else:
        data.append([ccy, round(sum(pnl)*10000), len(pnl), round(min(pnl)*10000), 
                     round(max(pnl)*10000),
                    round(sum(x>0 for x in pnl)/len(pnl),2), 
                    abs(round(sum(x for x in pnl if x>0)/sum(x for x in pnl if x<0),2))])
    
summary = pd.DataFrame(data, columns=['CCY pair', 'total pnl (in pips)', 
                                      'Number of trades', 
                                      'max drawdown (in pips)', 'max gain (in pips)', 'win%','gain/loss'])
display(summary)

,CCY pair,total pnl (in pips),Number of trades,max drawdown (in pips),max gain (in pips),win%,gain/loss
0,JPY=,2618.6,258,-162.0,538.0,0.47,1.81
1,AUDJPY=,2306.4,274,-105.5,365.5,0.49,2.07
2,NZDJPY=,2120.0,237,-104.0,390.5,0.54,2.26
3,EURJPY=,2160.0,276,-219.0,388.5,0.49,1.60
4,GBPJPY=,3610.5,298,-126.0,593.0,0.49,1.83
5,CADJPY=,-300.0,8,-270.0,113.0,0.50,0.51
6,CHFJPY=,865.0,127,-169.0,311.0,0.53,1.41
7,AUD=,874.0,283,-51.0,109.0,0.49,1.74
8,NZD=,516.0,308,-41.0,136.0,0.46,1.37
9,EUR=,520.0,270,-57.0,119.0,0.44,1.36
